In [1]:
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

In [2]:
df = pd.read_csv('/Users/jubba/Downloads/2_cleaning.csv')

In [3]:
df.columns

Index(['meeting_id', 'date', 'event_number', 'distance',
       'distance_raw_furlongs', 'total_prize_money', 'track_type', 'barrier',
       'number', 'finish_position', 'win_or_lose', 'failed_to_finish_reason',
       'margin', 'gear', 'jockey_allowance', 'handicap_weight', 'jockey_id',
       'tainer_id', 'dslr', 'horse_id', 'age', 'birth_date', 'official rating',
       'wfa', 'weight_adjustment', 'betfair_starting_price',
       'pre_race_master_rating_int', 'starting_price', 'track_condition_FAST',
       'track_condition_FIRM', 'track_condition_GOOD',
       'track_condition_GOOD TO FIRM', 'track_condition_GOOD TO SOFT',
       'track_condition_GOOD TO YIELDING', 'track_condition_HEAVY',
       'track_condition_SLOW', 'track_condition_SOFT',
       'track_condition_STANDARD', 'track_condition_STANDARD TO FAST',
       'track_condition_STANDARD TO SLOW', 'race_type_FLAT',
       'race_type_NATIONAL_HUNT_FLAT', 'race_class_1', 'race_class_2',
       'race_class_3', 'race_class_4',

In [4]:
X =df.drop(columns=['win_or_lose', 'date', 'birth_date', 'finish_position', 'failed_to_finish_reason'])
y=df['win_or_lose']

In [5]:
y = y.replace({0.5 : 1})

In [6]:
y.nunique()

2

In [7]:
X.columns

Index(['meeting_id', 'event_number', 'distance', 'distance_raw_furlongs',
       'total_prize_money', 'track_type', 'barrier', 'number', 'margin',
       'gear', 'jockey_allowance', 'handicap_weight', 'jockey_id', 'tainer_id',
       'dslr', 'horse_id', 'age', 'official rating', 'wfa',
       'weight_adjustment', 'betfair_starting_price',
       'pre_race_master_rating_int', 'starting_price', 'track_condition_FAST',
       'track_condition_FIRM', 'track_condition_GOOD',
       'track_condition_GOOD TO FIRM', 'track_condition_GOOD TO SOFT',
       'track_condition_GOOD TO YIELDING', 'track_condition_HEAVY',
       'track_condition_SLOW', 'track_condition_SOFT',
       'track_condition_STANDARD', 'track_condition_STANDARD TO FAST',
       'track_condition_STANDARD TO SLOW', 'race_type_FLAT',
       'race_type_NATIONAL_HUNT_FLAT', 'race_class_1', 'race_class_2',
       'race_class_3', 'race_class_4', 'race_class_5', 'race_class_6',
       'race_class_7', 'race_class_normalised_Claiming',


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

In [9]:
#xgbclassifierwere going to use becausewhethera horse loses or wins is biunary classificatiojn

In [10]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((145049, 60), (62165, 60), (145049,), (62165,))

In [11]:
model = xgb.XGBClassifier(
    tree_method='hist',
    booster='gbtree',
    objective='reg:logistic',
    random_state=42,
    learning_rate=0.001,
    colsample_bytree=0.9,
    n_estimators=110,
    subsample=0.75
    )

In [12]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.001, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=110, n_jobs=None,
              num_parallel_tree=None, objective='reg:logistic', ...)

In [13]:
import numpy as np

y_pred = model.predict(X_test)
np.unique(y_pred)

array([0])

In [14]:
accuracy = accuracy_score(y_test, y_pred)
accuracy


0.8873642725006032

In [15]:
from sklearn.model_selection import cross_validate

In [16]:
cv_results = cross_validate(model, X_train, y_train, cv=5)

In [17]:
cv_results

{'fit_time': array([0.523736  , 0.40163112, 0.49232388, 0.50118303, 0.41079521]),
 'score_time': array([0.01137996, 0.00991416, 0.00970411, 0.01060009, 0.01126385]),
 'test_score': array([0.88834885, 0.88834885, 0.88834885, 0.88831437, 0.888345  ])}

In [18]:
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

In [19]:
param_grid = {
    'tree_method': ['exact', 'hist'],
    'booster': ['gbtree', 'dart'],
    'learning_rate' : [0.001, 0.01],
            }

In [20]:
'''es = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0,
    patience=5,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0
        )'''

"es = EarlyStopping(\n    monitor='val_accuracy',\n    min_delta=0,\n    patience=5,\n    verbose=0,\n    mode='auto',\n    baseline=None,\n    restore_best_weights=True,\n    start_from_epoch=0\n        )"

In [34]:
xgb_classifier = xgb.XGBClassifier(objective='reg:logistic', random_state=42)
grid_search = GridSearchCV(estimator=xgb_classifier, refit='precision', param_grid= param_grid, cv=5, scoring=['accuracy', 'precision', 'recall'], n_jobs=-1, verbose=3)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/Users/jubba/.pyenv/versions/3.10.6/envs/which_horse/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jubba/.pyenv/versions/3.10.6/envs/which_horse/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jubba/.pyenv/versions/3.10.6/envs/which_horse/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

[CV 5/5] END booster=gbtree, learning_rate=0.001, tree_method=hist; accuracy: (test=0.888) precision: (test=0.000) recall: (test=0.000) total time=   1.6s
[CV 4/5] END booster=gbtree, learning_rate=0.001, tree_method=hist; accuracy: (test=0.888) precision: (test=0.000) recall: (test=0.000) total time=   1.7s
[CV 3/5] END booster=gbtree, learning_rate=0.001, tree_method=hist; accuracy: (test=0.888) precision: (test=0.000) recall: (test=0.000) total time=   1.7s
[CV 2/5] END booster=gbtree, learning_rate=0.001, tree_method=hist; accuracy: (test=0.888) precision: (test=0.000) recall: (test=0.000) total time=   1.8s
[CV 1/5] END booster=gbtree, learning_rate=0.001, tree_method=hist; accuracy: (test=0.888) precision: (test=0.000) recall: (test=0.000) total time=   1.8s
[CV 1/5] END booster=gbtree, learning_rate=0.01, tree_method=hist; accuracy: (test=0.996) precision: (test=0.978) recall: (test=0.988) total time=   1.6s
[CV 2/5] END booster=gbtree, learning_rate=0.01, tree_method=hist; accu

/Users/jubba/.pyenv/versions/3.10.6/envs/which_horse/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jubba/.pyenv/versions/3.10.6/envs/which_horse/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[CV 3/5] END booster=gbtree, learning_rate=0.001, tree_method=exact; accuracy: (test=0.888) precision: (test=0.000) recall: (test=0.000) total time=  22.7s
[CV 5/5] END booster=gbtree, learning_rate=0.001, tree_method=exact; accuracy: (test=0.888) precision: (test=0.000) recall: (test=0.000) total time=  22.7s


...

/Users/jubba/.pyenv/versions/3.10.6/envs/which_horse/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[CV 3/5] END booster=dart, learning_rate=0.001, tree_method=hist; accuracy: (test=0.888) precision: (test=0.000) recall: (test=0.000) total time= 4.3min


/Users/jubba/.pyenv/versions/3.10.6/envs/which_horse/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[CV 4/5] END booster=dart, learning_rate=0.001, tree_method=hist; accuracy: (test=0.888) precision: (test=0.000) recall: (test=0.000) total time= 4.4min
[CV 1/5] END booster=dart, learning_rate=0.01, tree_method=hist; accuracy: (test=0.996) precision: (test=0.978) recall: (test=0.988) total time= 4.0min
[CV 2/5] END booster=dart, learning_rate=0.01, tree_method=hist; accuracy: (test=0.996) precision: (test=0.974) recall: (test=0.990) total time= 3.9min
[CV 3/5] END booster=dart, learning_rate=0.01, tree_method=hist; accuracy: (test=0.996) precision: (test=0.979) recall: (test=0.986) total time= 3.6min
[CV 4/5] END booster=dart, learning_rate=0.01, tree_method=hist; accuracy: (test=0.996) precision: (test=0.973) recall: (test=0.989) total time= 3.6min
[CV 5/5] END booster=dart, learning_rate=0.01, tree_method=hist; accuracy: (test=0.996) precision: (test=0.976) recall: (test=0.989) total time= 3.6min


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     objective='reg:logistic', ...),
             n_jobs=-1,
             param_grid={'booster': ['gbtree', 'dart'],
                         'learning_rate': [0.001, 0.01],
                         'tree_method': ['exact', 'hist']},
             refit='precision', scoring=['accuracy', 'precision', 'recall'],
             verbose=3)

In [43]:
'''
param_grid = {
    'tree_method': ['exact', 'approx', 'hist'],
    'booster': ['gbtree', 'dart'],
    'learning_rate' : [0.001, 0.01, 0.1],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'n_estimators': [100, 150, 200],
    'subsample': [0.7, 0.75, 0.8]
            }'''

"\nparam_grid = {\n    'tree_method': ['exact', 'approx', 'hist'],\n    'booster': ['gbtree', 'dart'],\n    'learning_rate' : [0.001, 0.01, 0.1],\n    'colsample_bytree': [0.8, 0.9, 1.0],\n    'n_estimators': [100, 150, 200],\n    'subsample': [0.7, 0.75, 0.8]\n            }"

In [51]:
best_params = f"best_params {grid_search.best_params_}"
best_score = f"best_score {grid_search.best_score_}"
cv_results = grid_search.cv_results_

In [52]:
best_params

"best_params {'booster': 'gbtree', 'learning_rate': 0.01, 'tree_method': 'exact'}"

In [53]:
best_score

'best_score 0.9760909415739553'

In [54]:
cv_results

{'mean_fit_time': array([ 23.71115322,   1.64435973,  25.06086516,   1.42902131,
         56.36529317, 259.8737062 ,  58.34771585, 223.33002462]),
 'std_fit_time': array([ 0.96436945,  0.04567247,  0.3085668 ,  0.12016324,  1.75704057,
         0.47261534,  1.12486954, 11.10155684]),
 'mean_score_time': array([0.06157198, 0.06458688, 0.06287384, 0.06085525, 1.03406153,
        0.71242743, 0.89523897, 0.46057835]),
 'std_score_time': array([0.00931494, 0.00927424, 0.01264093, 0.00363337, 0.13719717,
        0.08998408, 0.10421476, 0.01487708]),
 'param_booster': masked_array(data=['gbtree', 'gbtree', 'gbtree', 'gbtree', 'dart', 'dart',
                    'dart', 'dart'],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_learning_rate': masked_array(data=[0.001, 0.001, 0.01, 0.01, 0.001, 0.001, 0.01, 0.01],
              mask=[False, False, False, False, False, False, False, False],
        fill_value

In [55]:
print(cv_results)

{'mean_fit_time': array([ 23.71115322,   1.64435973,  25.06086516,   1.42902131,
        56.36529317, 259.8737062 ,  58.34771585, 223.33002462]), 'std_fit_time': array([ 0.96436945,  0.04567247,  0.3085668 ,  0.12016324,  1.75704057,
        0.47261534,  1.12486954, 11.10155684]), 'mean_score_time': array([0.06157198, 0.06458688, 0.06287384, 0.06085525, 1.03406153,
       0.71242743, 0.89523897, 0.46057835]), 'std_score_time': array([0.00931494, 0.00927424, 0.01264093, 0.00363337, 0.13719717,
       0.08998408, 0.10421476, 0.01487708]), 'param_booster': masked_array(data=['gbtree', 'gbtree', 'gbtree', 'gbtree', 'dart', 'dart',
                   'dart', 'dart'],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_learning_rate': masked_array(data=[0.001, 0.001, 0.01, 0.01, 0.001, 0.001, 0.01, 0.01],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
         

In [56]:
cv_results.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_booster', 'param_learning_rate', 'param_tree_method', 'params', 'split0_test_accuracy', 'split1_test_accuracy', 'split2_test_accuracy', 'split3_test_accuracy', 'split4_test_accuracy', 'mean_test_accuracy', 'std_test_accuracy', 'rank_test_accuracy', 'split0_test_precision', 'split1_test_precision', 'split2_test_precision', 'split3_test_precision', 'split4_test_precision', 'mean_test_precision', 'std_test_precision', 'rank_test_precision', 'split0_test_recall', 'split1_test_recall', 'split2_test_recall', 'split3_test_recall', 'split4_test_recall', 'mean_test_recall', 'std_test_recall', 'rank_test_recall'])

In [60]:
cv_results['mean_test_recall']

array([0.        , 0.        , 0.98783646, 0.98845389, 0.        ,
       0.        , 0.98783646, 0.98845389])